# TODO：概要



# Connected Agents の主な制限

### 1. **ローカル関数は呼び出せない**

* **制限内容**
  　Connected Agents 機能を使って別のエージェントをツールとして追加した場合、その**サブエージェント側で定義した関数（Pythonのローカル関数）をFunction Tool経由で直接呼び出すことはできません**。
* **推奨される方法**
  　どうしても外部の処理を呼び出したい場合は、「OpenAPIツール」や「Azure Functions」を使って、API経由で実装することが推奨されています。

### 2. **出典（引用）の引き継ぎが保証されない**

* **制限内容**
  　Connected Agents経由で検索や回答をした際、**サブエージェントが生成した引用情報（例えばドキュメントの出典や根拠）が、親エージェントに必ず渡るとは限りません**。
* **対応策（完全ではない）**
  　プロンプトの工夫（Prompt Engineering）や、使うAIモデルを変えることで出典が返りやすくする方法もありますが、**必ず引用が親エージェントに引き継がれる保証はありません**。

### まとめ

* サブエージェントの**ローカル関数は直接使えない**ので、API公開（OpenAPIやAzure Functions）を利用しましょう。
* サブエージェントの**引用情報が必ず親エージェントまで引き継がれるわけではない**点も注意が必要です。

参考：[Limitations](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/connected-agents?pivots=python#limitations)

# ライブラリのインポート

In [59]:
import os
import time
import json
import datetime
import zoneinfo

import requests

from dotenv import load_dotenv, find_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    MessageTextContent,
    ListSortOrder,
    McpTool,
    MCPToolDefinition,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
    CodeInterpreterTool,
    FunctionTool,
    ToolSet,
    FilePurpose,
    FileSearchTool,
    RunAdditionalFieldList,
    RunStepFileSearchToolCall,
    RunStepToolCallDetails,
    ConnectedAgentTool
)


# 環境変数の取得

In [79]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")
FOUNDRY_CUSTOM_FUNCTIONS_AGENT_ID=os.getenv("FOUNDRY_CUSTOM_FUNCTIONS_AGENT_ID")
FOUNDRY_CODE_INTERPRETER_AGENT_ID=os.getenv("FOUNDRY_CODE_INTERPRETER_AGENT_ID")
FOUNDRY_MCP_AGENT_ID=os.getenv("FOUNDRY_MCP_AGENT_ID")
FOUNDRY_FILE_SEARCH_AGENT_ID=os.getenv("FOUNDRY_FILE_SEARCH_AGENT_ID")

# クライアントの初期化

In [61]:
# AI Project Client を初期化
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

# AgentClient の作成
agents_client = project_client.agents

# ユーティリティ関数

In [62]:
def agent_run_outputs(thread_id, agents_client, target_dir="./output_images"):
    """
    指定したスレッドIDのRun実行結果（テキスト・画像）をNotebook上に表示＆画像は保存。
    """
    messages = agents_client.messages.list(thread_id=thread_id, order=ListSortOrder.ASCENDING)
    os.makedirs(target_dir, exist_ok=True)

    for message in messages:
        # テキスト出力
        if message.text_messages:
            for txt in message.text_messages:
                print(f"{message.role.upper()}: {txt.text.value}")
        
        # 画像出力
        if hasattr(message, "image_contents"):
            for image_content in message.image_contents:
                file_id = image_content.image_file.file_id
                file_name = f"{file_id}_image_file.png"

                agents_client.files.save(
                    file_id=file_id,
                    file_name=file_name,
                    target_dir=target_dir
                )
                print(f"Saved image: {file_name}")
                display(Image(filename=f"{target_dir}/{file_name}"))

In [92]:
def file_search_content_outputs(agents_client, thread_id, run_id):
    """
    指定したスレッドID・Run IDのファイル検索ツールによる検索結果コンテンツをNotebook等に出力します。

    :param agents_client: AgentClient インスタンス
    :param thread_id: スレッドID
    :param run_id: Run ID
    """
    from azure.ai.agents.models import RunAdditionalFieldList, RunStepToolCallDetails, RunStepFileSearchToolCall

    for run_step in agents_client.run_steps.list(
        thread_id=thread_id,
        run_id=run_id,
        include=[RunAdditionalFieldList.FILE_SEARCH_CONTENTS]
    ):
        if isinstance(run_step.step_details, RunStepToolCallDetails):
            for tool_call in run_step.step_details.tool_calls:
                if (
                    isinstance(tool_call, RunStepFileSearchToolCall)
                    and getattr(tool_call, "file_search", None)
                    and getattr(tool_call.file_search, "results", None)
                    and tool_call.file_search.results
                    and getattr(tool_call.file_search.results[0], "content", None)
                    and tool_call.file_search.results[0].content
                    and getattr(tool_call.file_search.results[0].content[0], "text", None)
                ):
                    file_name = tool_call.file_search.results[0].file_name
                    content_text = tool_call.file_search.results[0].content[0].text
                    print(f"ファイル検索の結果、以下のコンテンツがヒットしました。\n「{file_name}」:")
                    print("=" * 60)
                    print(content_text)
                    print("=" * 60)

# Connected Agents の定義

In [81]:
# エージェントの取得
code_interpreter_agent = agents_client.get_agent(agent_id=FOUNDRY_CODE_INTERPRETER_AGENT_ID)
file_search_agent = agents_client.get_agent(agent_id=FOUNDRY_FILE_SEARCH_AGENT_ID)
mcp_agent = agents_client.get_agent(agent_id=FOUNDRY_MCP_AGENT_ID)

# ConnectedAgentTool の作成
code_interpreter_agent_tool = ConnectedAgentTool(
    id=code_interpreter_agent.id,
    name="code_interpreter_agent",
    description=(
        "Code Interpreter を利用して、計算や図表の出力に特化した分析アシスタントです。"
    ),
)

file_search_agent_tool = ConnectedAgentTool(
    id=file_search_agent.id,
    name="file_search_agent",
    description=(
        "File Search Agent を利用して、ファイルの検索や内容の取得を行うことができます。"
    ),
)

mcp_agent_tool = ConnectedAgentTool(
    id=mcp_agent.id,
    name="mcp_agent",
    description=(
        "あなたは Microsoft の公式ドキュメントを検索できる MCP ツールを持っているエージェントです。"
    ),
)

# エージェントの作成

In [ ]:
triage_agent  = agents_client.create_agent(
    model=AZURE_DEPLOYMENT_NAME,
    name="triage_agent ",
    instructions=(
        "あなたは振り分けエージェントです。"
        "接続されたエージェントを使用して質問に答えてください。"
        # " - custom_functions_agent: 日本の都道府県の地理情報（緯度・経度）や、日本標準時の日付、指定した場所・日付の気温データを自動的に取得するアシスタント"
        " - code_interpreter_agent: Code Interpreter を利用して、計算や図表の出力に特化した分析アシスタント"
        " - file_search_agent: File Search Agent を利用して、ファイルの検索や内容の取得を行うことができるアシスタント"
        # " - mcp_agent: Microsoft の公式ドキュメントを検索できる MCP ツールを持っているサポートアシスタント"
    ),
    tools=[
        code_interpreter_agent_tool.definitions[0],
        file_search_agent_tool.definitions[0],
        # mcp_agent_tool.definitions[0],
    ],
    temperature=0.1
)
print(f"Created Agent. AGENT_ID: {triage_agent.id}")


Created Agent. AGENT_ID: asst_VZlqy2Cpw0U95s2Ptrd9uQuL


In [ ]:
triage_agent_dict = triage_agent.as_dict()
print(json.dumps(triage_agent_dict, indent=2, ensure_ascii=False))

# スレッドの作成

In [89]:
# Thread の作成
thread = agents_client.threads.create()
print(f"Created Thread. THREAD_ID: {thread.id}")

Created Thread. THREAD_ID: thread_4np7A5IIrs381kaCgSr16bXs


# ユーザーメッセージの追加

In [90]:
# メッセージの追加
# user_message= (
#     "初期投資額が100万円、年利回りが3.5%、毎年10万円ずつ追加投資を"
#     "20年間継続したときの最終的な資産額を計算し、円単位で四捨五入して答えてください。"
# )
# user_message = (
#     "Azure AI Foundry の最新情報を教えてください。"
# )

user_message = (
    "新しいパソコンの購入を検討しています。しかし、初期設定に自信がないので、初期設定サポートについて教えてください。"
)

message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message,
)

print(f"Added Message. MESSAGE_ID: {message.id}")

Added Message. MESSAGE_ID: msg_L2iuBaOYa44wubbqbFiohMPA


# Run の実行

In [91]:
run = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=triage_agent.id
)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    agent_run_outputs(thread.id, agents_client)

USER: 新しいパソコンの購入を検討しています。しかし、初期設定に自信がないので、初期設定サポートについて教えてください。
ASSISTANT: パソコンの初期設定サポートについてご説明します。

### 初期設定サポートとは？
新しく購入したパソコンを使い始める前に必要な設定（初期設定）を、専門スタッフがサポートしてくれるサービスです。主に以下のような内容が含まれます。

#### 主なサポート内容
- **Windowsの初回セットアップ**（言語設定、ユーザーアカウント作成など）
- **インターネット接続設定**
- **メールソフトの設定**
- **プリンターや周辺機器の接続・設定**
- **セキュリティソフトのインストールと設定**
- **OSやメーカー提供のアップデート**
- 必要な**ソフトウェアのインストール**
- **データ移行**（オプションの場合あり）

#### サポートの提供形態
- **店頭サポート**：購入店でスタッフがその場で設定を行ってくれる
- **出張サポート**：スタッフが自宅まで訪問して設定を行う
- **リモートサポート**：インターネット経由で遠隔操作しながら設定を案内・実施

#### 費用
- 店舗やサービスによって異なります（約5,000円～15,000円程度が相場です）
- パソコン購入時に割安なパック料金を用意している場合もあります

#### 注意点
- データ移行や特別なソフトウェアのインストールは、追加料金がかかる場合があります。
- 自宅のネット環境やWi-Fiパスワードを事前に用意しておくと、サポートがスムーズです。

### 主な利用方法
- 購入時に「初期設定サポートを希望」と店舗スタッフに伝える
- または、購入後に家電量販店・パソコン専門店のサポート窓口や公式サイトから申し込む

---

何か特定のメーカーや購入予定店舗、具体的なサポート内容について知りたいことがあれば、詳細を教えてください。さらに詳しいサポート内容やおすすめの利用方法をご案内します。


In [94]:
# 参照したコンテンツを出力
file_search_content_outputs(agents_client, thread.id, run.id)

# Agent ID を .env ファイルに保存
※ 今回作成したエージェントを、後続の Connected Agents のハンズオン演習で使用するため永続化します。

In [ ]:
# 変数の定義
agent_env_key = "FOUNDRY_TRIAGE_AGENT_ID"
agent_env_value = triage_agent.id

# .envファイルのパスを自動探索
env_path = find_dotenv()  # 見つからなければ''を返す
if not env_path:
    raise FileNotFoundError(".envファイルが見つかりませんでした。")

# AGENT_ID を .env ファイルに追記
with open(env_path, "a", encoding="utf-8") as f:
    f.write(f'\n{agent_env_key}="{agent_env_value}"')

print(f'.envファイルに {agent_env_key}=\"{agent_env_value}\" を追記しました。')
